In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# readUpload
class readUpload:
    def __init__(self, filename, pose):
        self.cap = cv2.VideoCapture(filename)
        self.pose = pose
        
    # def is_opened(self):
    #     return self.cap.isOpened()
        
    # def read_frame(self):
    #     if self.cap.isOpened():
    #         ret, frame = self.cap.read()
    #         if ret is False or frame is None:
    #             return None
    #     else: 
    #         return None
    #     return frame
    
    def recolor_RGB(self, frame):
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        if frame is None: 
            return("a")
        else:
            results = self.pose.process(frame)
            return image, results
    
    # def recolor_BGR(self, image):
    #     image.flags.writeable = True
    #     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #     return image
    
    def calculate_angle(self,a,b,c):
        a = np.array(a) # First
        b = np.array(b) # Mid
        c = np.array(c) # End
    
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)
    
        if angle >180.0:
            angle = 360-angle
        
        return angle 
    
    def angle_of_singleline(self,point1, point2):
        """ Calculate angle of a single line """
        x_diff = point2[0] - point1[0]
        y_diff = point2[1] - point1[1]
        return math.degrees(math.atan2(y_diff, x_diff))

    

# Pushup pose
class Pushup(readUpload):
    def __init__(self, filename):
        super().__init__(filename)
        self.reps = 0 
        self.stage = ""
        self.hint = ""


    def angle(self, results, image):
        #print(landmark)
        if results.pose_landmarks is not None:
            landmarks = results.pose_landmarks.landmark

    # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y] 

            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

        # Calculate angle
            lKnee_angle = round(self.calculate_angle(left_hip, left_knee, left_ankle),2)
            rKnee_angle = round(self.calculate_angle(right_hip, right_knee, right_ankle),2)
            back_angle = round(self.angle_of_singleline(left_shoulder, left_hip),2)

            # Hip & Knee connection with the horizontal plane
            knee_angle = round(self.angle_of_singleline(left_hip, left_knee),2)


        # Visualize angle
            cv2.putText(image, str(lKnee_angle), 
                           tuple(np.multiply(left_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                                )
        
            cv2.putText(image, str(back_angle), 
                           tuple(np.multiply(right_knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
        # Curl counter logic. Add angle of the trailing knee to distinguish rear view (doesnt work yet, since detection sometimes gets confused when using side view)
            if lKnee_angle < 100 and rKnee_angle < 100 and back_angle > 75 and back_angle < 110: #and knee_angle > 50 and knee_angle < 140:
                self.stage = "down"
            if lKnee_angle > 160 and rKnee_angle > 160 and back_angle > 75 and back_angle < 110 and self.stage =='down':
                self.stage="up"
                self.reps +=1
                print(self.reps)
            if back_angle < 75 or back_angle > 110:
                self.hint = "Keep back straight"
            else:
                self.hint=""

        else:
            landmarks = None
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        cv2.rectangle(image, (930,0), (1280,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(self.reps), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, self.stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Hints & Feedback
        cv2.putText(image, 'HINT:', (950,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, self.hint, 
                    (945,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.1, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
        
    def visualize(self):
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image, results = self.recolor_RGB(frame)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            self.angle(results, image)

            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()


In [ ]:
# To test
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = Pushup(0, pose) 
cap.visualize()